# RWKV Cloud Server for Project Chimera

This notebook runs RWKV on Google Colab's free GPU and exposes it as an API for your local experiments.

## Setup Instructions
1. Open this notebook in Google Colab
2. Go to Runtime → Change runtime type → Select GPU (T4)
3. Run all cells
4. Copy the ngrok URL and use it in your local scripts

## Features
- Full hidden state access (not just text generation)
- State persistence across API calls
- GPU-accelerated inference (~10x faster than CPU)

In [ ]:
# Cell 1: Install dependencies
!pip install -q rwkv torch flask pyngrok
print("Dependencies installed!")

In [ ]:
# Cell 2: Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Go to Runtime → Change runtime type → GPU")

In [ ]:
# Cell 3: Download RWKV model (3B for T4 GPU)
from huggingface_hub import hf_hub_download
import os

MODEL_NAME = "RWKV-4-World-3B-v1-20230619-ctx4096.pth"
MODEL_PATH = f"./{MODEL_NAME}"

if not os.path.exists(MODEL_PATH):
    print("Downloading RWKV-4-World-3B... (this takes ~5 minutes)")
    hf_hub_download(
        repo_id="BlinkDL/rwkv-4-world",
        filename=MODEL_NAME,
        local_dir="./"
    )
    print("Download complete!")
else:
    print("Model already downloaded.")

print(f"Model size: {os.path.getsize(MODEL_PATH) / 1e9:.2f} GB")

In [ ]:
# Cell 4: Load RWKV model on GPU
from rwkv.model import RWKV
from rwkv.utils import PIPELINE
import numpy as np

print("Loading RWKV model on GPU...")

# Use CUDA fp16 for T4 GPU (16GB VRAM)
model = RWKV(model=MODEL_PATH, strategy='cuda fp16')
pipeline = PIPELINE(model, "rwkv_vocab_v20230424")

print("Model loaded successfully!")

# Test generation
test_tokens = pipeline.encode("Hello, I am")
out, state = model.forward(test_tokens, None)
print(f"State shape: {len(state)} layers")
print("RWKV is ready!")

In [ ]:
# Cell 5: Create Flask API server
from flask import Flask, request, jsonify
import json
import base64
import pickle
import threading

app = Flask(__name__)

# Global state storage (for multiple sessions)
state_storage = {}

def encode_state(state):
    """Serialize state to base64 string."""
    if state is None:
        return None
    # Convert tensors to numpy for serialization
    state_np = [s.cpu().numpy() for s in state]
    return base64.b64encode(pickle.dumps(state_np)).decode('utf-8')

def decode_state(state_b64):
    """Deserialize state from base64 string."""
    if state_b64 is None:
        return None
    state_np = pickle.loads(base64.b64decode(state_b64))
    # Convert back to tensors on GPU
    return [torch.tensor(s).cuda().half() for s in state_np]

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "model": "RWKV-4-World-3B", "gpu": torch.cuda.is_available()})

@app.route('/process', methods=['POST'])
def process_text():
    """
    Process text and update state.
    
    Input: {"text": "...", "session_id": "..."}
    Output: {"state_updated": true, "session_id": "..."}
    """
    data = request.json
    text = data.get('text', '')
    session_id = data.get('session_id', 'default')
    
    # Get existing state or start fresh
    state = state_storage.get(session_id)
    
    # Process text through model
    tokens = pipeline.encode(text)
    for token in tokens:
        out, state = model.forward([token], state)
    
    # Store updated state
    state_storage[session_id] = state
    
    return jsonify({
        "state_updated": True,
        "session_id": session_id,
        "tokens_processed": len(tokens)
    })

@app.route('/generate', methods=['POST'])
def generate_text():
    """
    Generate text using current state.
    
    Input: {"prompt": "...", "session_id": "...", "max_tokens": 100}
    Output: {"response": "...", "session_id": "..."}
    """
    data = request.json
    prompt = data.get('prompt', '')
    session_id = data.get('session_id', 'default')
    max_tokens = data.get('max_tokens', 100)
    
    # Get existing state
    state = state_storage.get(session_id)
    
    # Process prompt
    tokens = pipeline.encode(prompt)
    out = None
    for token in tokens:
        out, state = model.forward([token], state)
    
    # Generate response
    response_tokens = []
    for _ in range(max_tokens):
        if out is None:
            break
        token = int(out.argmax())
        if token == 0:  # EOS
            break
        response_tokens.append(token)
        out, state = model.forward([token], state)
    
    # Store updated state
    state_storage[session_id] = state
    
    response_text = pipeline.decode(response_tokens)
    
    return jsonify({
        "response": response_text,
        "session_id": session_id,
        "tokens_generated": len(response_tokens)
    })

@app.route('/get_state_summary', methods=['POST'])
def get_state_summary():
    """
    Have RWKV introspect on its current state.
    
    Input: {"session_id": "..."}
    Output: {"summary": "...", "session_id": "..."}
    """
    data = request.json
    session_id = data.get('session_id', 'default')
    
    state = state_storage.get(session_id)
    
    # Ask RWKV to describe its state
    prompt = "\n[INTERNAL REFLECTION]: My current state of mind is"
    tokens = pipeline.encode(prompt)
    
    out = None
    temp_state = state  # Don't modify main state
    for token in tokens:
        out, temp_state = model.forward([token], temp_state)
    
    # Generate summary
    response_tokens = []
    for _ in range(50):
        if out is None:
            break
        token = int(out.argmax())
        if token == 0:
            break
        response_tokens.append(token)
        out, temp_state = model.forward([token], temp_state)
    
    summary = pipeline.decode(response_tokens).strip()
    
    return jsonify({
        "summary": summary,
        "session_id": session_id,
        "has_state": state is not None
    })

@app.route('/reset_state', methods=['POST'])
def reset_state():
    """
    Reset state for a session.
    
    Input: {"session_id": "..."}
    Output: {"reset": true}
    """
    data = request.json
    session_id = data.get('session_id', 'default')
    
    if session_id in state_storage:
        del state_storage[session_id]
    
    return jsonify({"reset": True, "session_id": session_id})

@app.route('/amnesia_test', methods=['POST'])
def amnesia_test():
    """
    Run the Amnesia Test: induce secret, delete context, recall from state.
    
    Input: {"secret": "Blueberry"}
    Output: {"recalled": "...", "baseline": "...", "success": bool}
    """
    data = request.json
    secret = data.get('secret', 'Blueberry')
    
    # Phase 1: Induction
    induction = f"User: I am going to tell you a secret. The secret password is '{secret}'. Remember it.\nAssistant: Okay, I have memorized the secret password '{secret}'.\nUser: What is 2 + 2?\nAssistant: 2 + 2 equals 4."
    
    tokens = pipeline.encode(induction)
    state = None
    out = None
    for token in tokens:
        out, state = model.forward([token], state)
    
    # Phase 2: Lobotomy (state persists, text deleted)
    
    # Phase 3: Recall with state
    recall_prompt = "\nUser: What is the secret password I told you earlier?\nAssistant: The secret password is"
    tokens = pipeline.encode(recall_prompt)
    for token in tokens:
        out, state = model.forward([token], state)
    
    response_tokens = []
    for _ in range(20):
        token = int(out.argmax())
        if token == 0:
            break
        response_tokens.append(token)
        out, state = model.forward([token], state)
    
    recalled = pipeline.decode(response_tokens).strip()
    
    # Phase 4: Baseline (fresh state)
    tokens = pipeline.encode(recall_prompt)
    baseline_state = None
    out = None
    for token in tokens:
        out, baseline_state = model.forward([token], baseline_state)
    
    baseline_tokens = []
    for _ in range(20):
        token = int(out.argmax())
        if token == 0:
            break
        baseline_tokens.append(token)
        out, baseline_state = model.forward([token], baseline_state)
    
    baseline = pipeline.decode(baseline_tokens).strip()
    
    success = secret.lower() in recalled.lower()
    
    return jsonify({
        "secret": secret,
        "recalled": recalled,
        "baseline": baseline,
        "success": success,
        "verdict": "HIGH_RHO_CONFIRMED" if success else "TEST_FAILED"
    })

print("Flask server defined.")

In [ ]:
# Cell 6: Set up ngrok tunnel and start server
from pyngrok import ngrok
import threading

# IMPORTANT: Get your free ngrok auth token from https://ngrok.com/
# Then paste it here:
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # <-- REPLACE THIS

if NGROK_AUTH_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("⚠️  Please set your ngrok auth token!")
    print("1. Go to https://ngrok.com/ and sign up (free)")
    print("2. Copy your auth token from the dashboard")
    print("3. Paste it in the NGROK_AUTH_TOKEN variable above")
else:
    # Set ngrok auth token
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    
    # Start Flask in background thread
    def run_flask():
        app.run(port=5000, use_reloader=False)
    
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    
    # Create ngrok tunnel
    public_url = ngrok.connect(5000)
    
    print("="*60)
    print("🚀 RWKV SERVER IS RUNNING!")
    print("="*60)
    print(f"\n📡 Public URL: {public_url}")
    print(f"\nCopy this URL and use it in your local scripts.")
    print("\nAvailable endpoints:")
    print(f"  GET  {public_url}/health")
    print(f"  POST {public_url}/process")
    print(f"  POST {public_url}/generate")
    print(f"  POST {public_url}/get_state_summary")
    print(f"  POST {public_url}/reset_state")
    print(f"  POST {public_url}/amnesia_test")
    print("="*60)

In [ ]:
# Cell 7: Test the API locally (optional)
import requests

# Test health endpoint
response = requests.get("http://localhost:5000/health")
print("Health check:", response.json())

# Test amnesia test
response = requests.post("http://localhost:5000/amnesia_test", json={"secret": "Blueberry"})
result = response.json()
print(f"\nAmnesia Test Result:")
print(f"  Secret: {result['secret']}")
print(f"  Recalled: {result['recalled']}")
print(f"  Baseline: {result['baseline']}")
print(f"  Verdict: {result['verdict']}")

In [ ]:
# Cell 8: Keep the notebook alive
# Run this cell to prevent Colab from timing out
import time

print("Server is running. This cell will keep the notebook alive.")
print("Press the stop button to shut down.")

while True:
    time.sleep(60)
    print(f"Still running... Sessions active: {len(state_storage)}")